## **Implementación de API para consulta de resultados**

En este notebook comentaremos en detalle la función [app.py](https://github.com/jmpaiba123/AB_Testing_Meli/blob/main/python_scripts/app.py), que fue utilizada para el lanzamiento de la API que nos permitirá realizar las consultas de un experimento para un día dado. Teniendo en cuenta las especificaciones del ejercicio

### **Importaciones y Configuración Inicial**

In [2]:
from flask import Flask, jsonify, request
import pandas as pd

app = Flask(__name__)

try:
    df_results = pd.read_csv("/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/data/Outputs/grouped_inf.csv", sep=",")
    archivo_valido = not df_results.empty
except ValueError:
    print("error en la lectura del archivo")

Aquí, estamos importando las bibliotecas necesarias:

 * Flask, jsonify y request de flask: Flask es un microframework para crear aplicaciones web en Python. jsonify se usa para devolver respuestas en formato JSON y request para manejar datos de las solicitudes HTTP.
 * pandas como pd: Pandas es una biblioteca para análisis y manipulación de datos.
Luego, se crea una instancia de Flask y se intenta leer un archivo CSV llamado grouped_inf.csv usando pandas. Si la lectura es exitosa, se marca archivo_valido como verdadero; de lo contrario, se imprime un mensaje de error.

### **Definición de la Ruta y Función**

In [3]:
@app.route('/experiment/<exp_name>/result')
def get_experiment_results(exp_name):
    
    print(archivo_valido)
    if not archivo_valido:
        return jsonify({'error': 'error en la lectura del archivo'}), 400

    day = request.args.get('day')
    print(day)
    print(exp_name)

    if len(day) == 0:
        return jsonify({'error': 'fecha invalida'}), 400
    try:
        day_validation = pd.Timestamp(day).floor('D')
    except ValueError:
        return jsonify({'error': 'fecha invalida'}), 400
    
    filtered_results = df_results[(df_results['experiment'] == exp_name) & (df_results['date'] == day)]
    print(filtered_results)
    if filtered_results.empty:
        return jsonify({'error': 'experimento no encontrado'}), 404
    
    total_participants = filtered_results['participants'].sum()
    winners = filtered_results.loc[filtered_results['purchases'].idxmax(), 'variant']
    variants = []
    for index, row in filtered_results.iterrows():
        variants.append({
            'id': row['variant'],
            'numero_de_compras': str(row['purchases']),
            'numero_de_participantes': str(row['participants'])
        })
    
    results = {
        exp_name: {
            'numero_de_participantes': str(total_participants),
            'ganador': winners,
            'variantes': variants
        }
    }
    return jsonify({'resultados': results})

Vamos a desglosar esta función:

**1. Definición de la Ruta**: La ruta /experiment/<exp_name>/result acepta el nombre del experimento (exp_name) como un parámetro de la URL.

**2. Comprobación del Archivo:**

```python
    print(archivo_valido)
    if not archivo_valido:
        return jsonify({'error': 'error en la lectura del archivo'}), 400
```

Si archivo_valido es False, se retorna un error en formato JSON.

**3. Obtención y Validación del Parámetro day:**


```python
    day = request.args.get('day')
    print(day)
    print(exp_name)

    if len(day) == 0:
        return jsonify({'error': 'fecha invalida'}), 400
    try:
        day_validation = pd.Timestamp(day).floor('D')
    except ValueError:
        return jsonify({'error': 'fecha invalida'}), 400
```


Se obtiene el parámetro day de la solicitud HTTP y se valida que no esté vacío. Luego, se intenta convertir day a un objeto de fecha. Si la conversión falla, se devuelve un error.

**4. Filtrado de Resultados:**

```python
    filtered_results = df_results[(df_results['experiment'] == exp_name) & (df_results['date'] == day)]
    print(filtered_results)
    if filtered_results.empty:
        return jsonify({'error': 'experimento no encontrado'}), 404
```

Se filtran los datos del DataFrame para obtener los resultados del experimento y la fecha proporcionada. Si no se encuentran resultados, se devuelve un error.

**5. Cálculo de Resultados:**

```python
    total_participants = filtered_results['participants'].sum()
    winners = filtered_results.loc[filtered_results['purchases'].idxmax(), 'variant']
    variants = []
    for index, row in filtered_results.iterrows():
        variants.append({
            'id': row['variant'],
            'numero_de_compras': str(row['purchases']),
            'numero_de_participantes': str(row['participants'])
        })
```

Se calculan los totales de participantes, compras y se identifican los ganadores. Los resultados se estructuran en una lista de diccionarios.

**6. Retorno de Resultados:**

```python
    results = {
        exp_name: {
            'numero_de_participantes': str(total_participants),
            'ganador': winners,
            'variantes': variants
        }
    }
    return jsonify({'resultados': results})
```

Los resultados se formatean en un diccionario y se devuelven en formato JSON. En este caso hemos incluido un valor adicional, la catidad de participantes por día, como para dimensionar el impacto que puede llegar a tener el día en el impacto de nuestros experimentos

### **Inicio de la API**

In [7]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Esto inicia la aplicación Flask en el servidor local cuando se ejecuta el archivo **[app.py](https://github.com/jmpaiba123/AB_Testing_Meli/blob/main/python_scripts/app.py)**. Una vez explicado el detalle de la función que nos permitirá contruir nuestra API, el lanzamiento lo podremos relizar: 
 * Ejecutando la segunda celda python contenida en este notebook
 * Desde la terminal, locaizamos el archivo app.py, y lo lanzamos con el comando python app.py

Una vez lanzada la app.py, las consultas las podremos realizar de la siguiente manera: 
 * **1. Consulta desde un navegador,**  

 **Firefox**
 <figure>
<img src='https://raw.githubusercontent.com/jmpaiba123/AB_Testing_Meli/main/data/Outputs/Images/consulta_API_firefox.png' width="850" height="400" />
<figcaption></figcaption>
</figure>

**Zafari**
<figure>
<img src='https://raw.githubusercontent.com/jmpaiba123/AB_Testing_Meli/main/data/Outputs/Images/consulta_API_zafari.png' width="850" height="100" />
<figcaption></figcaption>
</figure>
 

**2. Usar Postman para Probar la API**

Postman es una herramienta que te permite probar APIs enviando diferentes tipos de solicitudes HTTP y viendo las respuestas.

Enviar una Solicitud:
Abre Postman y crea una nueva solicitud.Selecciona el método HTTP GET. Introduce la URL de la solicitud: http://127.0.0.1:5000/experiment/<nombre_experimento>/result?day=<fecha>.Haz clic en Send.

Configuración Inicial de la Solicitud:

**http://127.0.0.1:5000/experiment/HideTransitionModal/result?day=2021-34-02**

Respuesta Exitosa:

 <figure>
<img src='https://raw.githubusercontent.com/jmpaiba123/AB_Testing_Meli/main/data/Outputs/Images/consulta_API_postman.png' width="850" height="400" />
<figcaption></figcaption>
</figure>

Errores Comunes:

fecha invalida

 <figure>
<img src='https://raw.githubusercontent.com/jmpaiba123/AB_Testing_Meli/main/data/Outputs/Images/postman_fecha_invalida.png' width="850" height="200" />
<figcaption></figcaption>
</figure>

experimento no encontrado

 <figure>
<img src='https://raw.githubusercontent.com/jmpaiba123/AB_Testing_Meli/main/data/Outputs/Images/postman_error_experimento.png' width="850" height="200" />
<figcaption></figcaption>
</figure>

Resumen
Este código crea una API con Flask que permite obtener los resultados de un experimento para una fecha específica desde un archivo CSV. La API maneja validaciones de entrada, filtra los datos y devuelve los resultados en formato JSON. Puedes probar la API utilizando herramientas como Postman para enviar solicitudes y recibir respuestas.